In [ ]:
import os
import urllib

import contextily as ctx
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import requests
import seaborn as sns
from shapely.wkt import loads

In [ ]:
# get data from datasette
datasette_base_url = "https://datasette.planning.data.gov.uk/conservation-area.csv"

query = """
select * 
from entity
"""
encoded_query = urllib.parse.urlencode({"sql": query})

r = requests.get(f"{datasette_base_url}?{encoded_query}", auth=("user", "pass"))

filename = "datasette_data.csv"
with open(filename, "wb") as f_out:
    f_out.write(r.content)

In [ ]:
data = pl.read_csv(filename)

In [ ]:
data[1]

In [ ]:
type(data["geometry"][0]), data["geometry"][0]

In [ ]:
multipolygon_geom = loads(data["geometry"][1])

In [ ]:
multipolygon_geom

In [ ]:
data[3]

In [ ]:
multipolygon_geom = loads(data["geometry"][100])
geometries = [multipolygon_geom]
data_point = {"id": [1]}

gdf = gpd.GeoDataFrame(data_point, geometry=geometries, crs="EPSG:4326")

gdf_web_mercator = gdf.to_crs(epsg=3857)

fig = plt.figure()

# ax = gdf_web_mercator.plot(figsize=(10, 10), alpha=0.5, edgecolor="k")
# ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
# ax.set_axis_off()

# gdf_web_mercator = gdf.to_crs(epsg=3857)

ax = gdf_web_mercator.plot(figsize=(10, 10), alpha=0.5, edgecolor="k")
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
# ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

ax.set_axis_off()
plt.show()

In [ ]:
# multipolygon_geom.area

In [ ]:
m = gdf.explore(
    tooltip="id",  # Show the 'id' column on hover/click
    tiles="OpenStreetMap",
    style_kwds=dict(color="blue", weight=2, opacity=0.7),
)

In [ ]:
m